In [2]:
import os
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import decode_image

In [3]:
class CustomImageDataset(Dataset):
  def __init__(self,annotations_file,img_dir,tranform = None,target_transform = None):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.tranform = tranform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self,idx):
    img_path = os.path.join(self.img_dir,self.img_labels.iloc[idx,0])
    image = decode_image(img_path)
    label = self.img_labels.iloc[idx,1]
    if self.tranform:
      image = self.tranform(image)
    if self.target_transform:
      label = self.target_transform(label)
    return image,label

In [6]:
train_datasetloader = CustomImageDataset("C:/AI_ML_fresh/pytorch/annotations - Sheet1.csv","C:/AI_ML_fresh/pytorch/train")

In [7]:
train_datasetloader.__len__()

2

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### load dataset from the library

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])
trainset = torchvision.datasets.MNIST(root='./data',train = True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 64,shuffle = True)
testset = torchvision.datasets.MNIST(root='./data',train = False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size = 64,shuffle = False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 479kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.49MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.25MB/s]


In [9]:
class SimpleDNN(nn.Module):
  def __init__(self):
    super(SimpleDNN,self).__init__()
    self.fc1 = nn.Linear(28*28,256)
    self.fc2 = nn.Linear(256,128)
    self.fc3 = nn.Linear(128,10)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.2)

  def forward(self,x):
    x = x.view(-1,28*28)  # flatten the image
    x = self.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleDNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)
print(device)

cuda


In [13]:
epochs = 5
for epoch in range(epochs):
  running_loss = 0.0
  for images,labels in trainloader:
    images,labels = images.to(device),labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss+= loss.item()
  print(f"Epoch [{epoch+1}/{epochs}],loss: {running_loss/len(trainloader):.4f}")

Epoch [1/5],loss: 0.0951
Epoch [2/5],loss: 0.0931
Epoch [3/5],loss: 0.0865
Epoch [4/5],loss: 0.0845
Epoch [5/5],loss: 0.0791


### Evaluate Model

In [18]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
  for images,labels in  testloader:
    images,labels = images.to(device),labels.to(device)
    outputs = model(images)
    _,predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print(f"test accuracy : {100*correct /total:.2f}%")

test accuracy : 97.18%
